In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.common.exceptions import *
from datetime import datetime

In [2]:
ruta = "C:/Users/9 ----- SIG/Downloads/RG_003 delete.xlsx"
df = pd.read_excel(ruta)

### CODIGO PARA ELIMINAR GRADUADOS

In [ ]:
# Crear el navegador
driver = webdriver.Chrome()
try:
    # Login y navegación inicial
    driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
    time.sleep(2)

    # Login
    usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
    usuario.send_keys("1826-admin")
    contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
    contrasena.send_keys("1826Uan9123")
    boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
    boton_ingresar.click()

    # Navegación al menú
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "menu_129"))
    ).click()
    time.sleep(0.5)

    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "menu_135"))
    ).click()

    # Seleccionar tipo de documento
    dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "filter:tipoDoc_label"))
    )
    dropdown.click()
    time.sleep(0.5)

    opcion = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//li[@data-label='Cédula de ciudadanía']"))
    )
    opcion.click()
    time.sleep(0.3)

    # Variables para tracking
    procesadas = 0
    exitosas = 0
    errores = []

    # BUCLE PRINCIPAL - Procesar cada cédula
    for index, row in df.iterrows():
        cedula = str(row["DOCUMENTO"])
        print(f"Procesando cédula {procesadas + 1}/{len(df)}: {cedula}")
    
        try:
            # --- INGRESAR CÉDULA CON REINTENTOS PARA STALE ELEMENTS ---
            for attempt_cedula in range(3):
                try:
                    input_cedula = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.ID, "filter:numDoc"))
                    )
                    input_cedula.clear()
                    input_cedula.send_keys(cedula)
                    
                    boton_buscar = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']"))
                    )
                    boton_buscar.click()
                    break
                except StaleElementReferenceException:
                    print(f"Intento {attempt_cedula+1}/3: Elemento stale en búsqueda, reintentando...")
                    time.sleep(0.3)
                    if attempt_cedula == 2:
                        raise
    
            time.sleep(0.3)
            
            # Clic en "Consultar"
            codigo_programa = str(row["CODIGO_PROGRAMA"])
            fila_codigo = WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.XPATH, f"//td[contains(text(), '{codigo_programa}')]/parent::tr"))
            )
            boton_consultar = fila_codigo.find_element(By.TAG_NAME, "a")
            boton_consultar.click()
        
            time.sleep(0.3)
                
            # --- VERIFICACIÓN Y ELIMINACIÓN DE GRADUADO ---
            try:
                # Intentar encontrar la papelera
                papelera = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".circle .small-circle.color-border-lightblue ~ a i.fa.fa-trash"))
                )
                
                # --- SI EXISTE LA BURBUJA ---
                papelera.click()
                time.sleep(0.3)

                # Menú desplegable
                flecha_menu = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".ui-selectonemenu-trigger.ui-state-focus"))
                )
                flecha_menu.click()
                time.sleep(0.3)
                
                # --- SELECCIÓN ROBUSTA DE "ERROR DE REPORTE" ---
                try:
                    # Estrategia 1: Selección estándar
                    opcion_error_reporte = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.ID, "formDelete:justificacion2_2"))
                    )
                    opcion_error_reporte.click()
                    
                except (TimeoutException, ElementClickInterceptedException):
                    try:
                        # Estrategia 2: Búsqueda por texto
                        opcion_error_reporte = WebDriverWait(driver, 2).until(
                            EC.element_to_be_clickable((By.XPATH, "//li[contains(@data-label, 'Error de reporte')]"))
                        )
                        driver.execute_script("arguments[0].scrollIntoView(true);", opcion_error_reporte)
                        driver.execute_script("arguments[0].click();", opcion_error_reporte)
                        
                    except TimeoutException:
                        try:
                            # Estrategia 3: JavaScript directo
                            driver.execute_script("""
                                var items = document.querySelectorAll("li.ui-selectonemenu-item");
                                for (var i = 0; i < items.length; i++) {
                                    if (items[i].textContent.includes("Error de reporte")) {
                                        items[i].click();
                                        break;
                                    }
                                }
                            """)
                            time.sleep(0.3)
                            
                            # Verificar selección
                            if not driver.find_element(By.ID, "formDelete:justificacion2_2").is_selected():
                                raise Exception("No se pudo seleccionar 'Error de reporte'")
                                
                        except Exception as js_error:
                            print(f"⚠ Error seleccionando opción: {js_error}")
                            driver.refresh()
                            raise
                
                time.sleep(0.3)
    
                # --- ELIMINAR GRADUADO CON MANEJO DE OVERLAY ---
                try:
                    # Esperar a que el botón esté disponible
                    eliminar_btn = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//span[contains(., 'Eliminar Graduado')]"))
                    )
                    
                    # Hacer clic con JavaScript para evitar intercepciones
                    driver.execute_script("arguments[0].click();", eliminar_btn)
                    
                    # Esperar confirmación de eliminación
                    WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//div[contains(text(), 'eliminado exitosamente')]"))
                    )
                    
                except Exception as e:
                    print(f"Error al eliminar: {str(e)}")
                    raise
    
                # --- VOLVER CON MANEJO DE OVERLAY ---
                for attempt_volver in range(3):
                    try:
                        # Esperar a que desaparezca el overlay
                        WebDriverWait(driver, 1).until(
                            EC.invisibility_of_element_located((By.ID, "loadingDialogId_modal"))
                        )
                        
                        # Localizar botón Volver
                        volver_btn = WebDriverWait(driver, 1).until(
                            EC.element_to_be_clickable((By.XPATH, "//span[text()='Volver']"))
                        )
                        
                        # Hacer clic con JavaScript
                        driver.execute_script("arguments[0].click();", volver_btn)
                        
                        # Verificar que volvimos a la página de búsqueda
                        WebDriverWait(driver, 1).until(
                            EC.presence_of_element_located((By.ID, "filter:numDoc"))
                        )
                        break
                        
                    except (ElementClickInterceptedException, TimeoutException):
                        print(f"Intento {attempt_volver+1}/3: Overlay bloqueando, reintentando...")
                        
                        # Intentar ocultar el overlay manualmente
                        if driver.find_elements(By.ID, "loadingDialogId_modal"):
                            driver.execute_script("""
                                try {
                                    document.getElementById('loadingDialogId_modal').style.display = 'none';
                                } catch(e) {}
                            """)
                        time.sleep(0.2)
                        
                        if attempt_volver == 2:
                            # Último recurso: usar navegación hacia atrás
                            driver.back()
                            time.sleep(0.2)
                
                exitosas += 1
                print(f"✓ Eliminado: {cedula}")
    
            except TimeoutException:
                # --- NO EXISTE BURBUJA DE GRADUADO ---
                print(f"✗ No hay graduado: {cedula}")
                errores.append(f"{cedula}: Sin burbuja graduado")
                
                # Volver a la pantalla de búsqueda
                try:
                    WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.XPATH, "//span[text()='Volver']"))
                    ).click()
                except:
                    try:
                        driver.back()
                    except:
                        pass
                time.sleep(0.2)
    
        except Exception as e:
            print(f"Error procesando cédula {cedula}: {str(e)}")
            errores.append(f"{cedula}: {str(e)}")
    
        finally:
            procesadas += 1
            time.sleep(0.2)  # Pausa entre cédulas

    # Resumen final
    print(f"\n=== RESUMEN ===")
    print(f"Total procesadas: {procesadas}")
    print(f"Exitosas: {exitosas}")
    print(f"Con errores: {len(errores)}")

    if errores:
        print(f"\nErrores encontrados:")
        for error in errores:
            print(f"- {error}")
finally:
    # Cerrar el navegador
    input("Presiona Enter para cerrar el navegador...")
    driver.quit()